In [6]:
import sys
sys.path.append("..")
import importlib
import numpy as np
import matplotlib.pyplot as plt
import qutip
import scipy as sp
from tqdm import tqdm as tqdm
from qutip.parallel import parallel_map as parallel_map
from qutip.parallel import parfor as parfor

import system.from_higher_basis_truncation_tunable_full_cosine as system
importlib.reload(system)
import utils.floquet_solver as fsolver
importlib.reload(fsolver)
import utils.solver as solver
importlib.reload(solver)
import pulse.tanh as tanh
importlib.reload(tanh)
import utils.expectation_values as expv
importlib.reload(expv)
import utils.floquetify as floqify
importlib.reload(floqify)
import utils.graph as graph
importlib.reload(graph)
pass

# Generate a Landscape

In [7]:
amplitude = 7.034040495797981
guess_time = 146.17831127951132
off = 0.10188603062894215
freq = system.first_energy_spacing/3 - off

t_multiplier = 5
ramp_slope = 0.5
cut_factor = 3

def inner(dur):
    tlist = np.linspace(0, dur, int(dur*t_multiplier))
    tanh.setup(amplitude=amplitude, drive_frequency=freq, ramp_slope=ramp_slope, cut_factor=cut_factor, tlist=tlist)
    s = solver.time_evolve(system.H0, system.H1, tanh.pulse_func, tlist, system.starting_state)
    fidelity = expv.expectation_value(s.states[-1], system.target_state)
    return fidelity

def generate_fidelity_landscape(_amplitude, _ramp_slope, freq_list, dur_list, _t_multiplier=5, skip_rate=1):
    global t_multiplier, amplitude, freq, ramp_slope
    t_multiplier = _t_multiplier
    amplitude = _amplitude
    ramp_slope = _ramp_slope
    fids = [] # an array of columns in the landscape
    freq_points = np.linspace(np.amin(freq_list), np.amax(freq_list), int(len(freq_list)/skip_rate))
    dur_points = np.linspace(np.amin(dur_list), np.amax(dur_list), int(len(dur_list)/skip_rate))
    for f in tqdm(freq_points):
        freq = f
        fids.append(parallel_map(inner, dur_points))

    fids = np.swapaxes(fids,0,1)

    plt.pcolormesh(freq_points, dur_points, fids, shading='auto')
    plt.colorbar()
    plt.xlabel("frequency (GHz)")
    plt.ylabel("duration (ns)")
    
    return fids

In [8]:
freq_list = np.linspace(system.first_energy_spacing/3 - off*2, system.first_energy_spacing/3 + off*2, 50)
fids0 = generate_fidelity_landscape(amplitude, 0.5, freq_list, np.linspace(5, 200, 50))

 22%|██▏       | 11/50 [07:32<26:40, 41.04s/it]Process ForkPoolWorker-46:
Process ForkPoolWorker-47:
Process ForkPoolWorker-48:
Process ForkPoolWorker-45:
capi_return is NULL
Call-back cb_f_in_zvode__user__routines failed.
capi_return is NULL
capi_return is NULL
Call-back cb_f_in_zvode__user__routines failed.
Call-back cb_f_in_zvode__user__routines failed.
capi_return is NULL
Call-back cb_f_in_zvode__user__routines failed.
 22%|██▏       | 11/50 [07:35<26:55, 41.42s/it]


KeyboardInterrupt: 

## Define points for floquet analysis

In [ ]:
points = [()] # [ (frequency, duration) ]
freq_points = [freq for freq, dur in points]
dur_points = [dur for freq, dur in points]

In [ ]:
plt.pcolormesh(freq_list, np.linspace(5, 200, 50), fids0, shading="auto")
plt.scatter(dur_points, freq_points, c="red")

# Look at first point

In [ ]:
point_index = 0
point_freq = freq_points[point_index]
point_dur = dur_points[point_index]

plt.pcolormesh(freq_list, np.linspace(5, 200, 50), fids0, shading="auto")
plt.scatter(point_dur, point_freq, c="red")
plt.plot()

tlist = np.linspace(0, point_dur, int(point_dur*10))
tanh.setup(amplitude, point_freq, ramp_slope, cut_factor, tlist)
plt.plot(tlist, tanh.get_pulse(tlist))
plt.xlabel("duration (ns)")
plt.ylabel("pulse amplitude")
plt.show()

solution = solver.time_evolve(system.H0, system.H1, tanh.pulse_func, tlist, system.starting_state)
graph.graph_solution(solution, system)
plt.plot()
floquet_results = floqify.floquetify(system, solution, tanh, tlist)